In [27]:
from datasets import load_dataset
from transformers import AutoImageProcessor, ResNetModel
import torch

# Load the dataset
dataset = load_dataset("frgfm/imagenette", 'full_size')

# Load the image processor
# image_processor = AutoImageProcessor.from_pretrained("./resnet-50", local_files_only=True)

# # Function to apply preprocessing
# def preprocess_images(examples):
#     # Process images and convert them to tensors
#     examples['pixel_values'] = [ image_processor(images=image).pixel_values for image in examples['image']]
#     return examples

# # Apply preprocessing to all images in the dataset
# dataset = dataset.map(preprocess_images, batched=True, remove_columns=['image'])

# # Example of how to access a preprocessed batch
# batch = dataset['train'][:10]
# inputs = torch.stack(batch['pixel_values'])  # Stack to create a batch tensor



In [28]:
from torchvision import transforms

# Define the PyTorch transform pipeline
transform = transforms.Compose([
    transforms.Resize(256),  # Resize such that the shortest side is 256 pixels
    transforms.CenterCrop(224),  # Crop the image at the center to 224x224
    transforms.ToTensor(),  # Convert image to a tensor
    transforms.ConvertImageDtype(torch.float),  # Ensure the tensor is of type float
    transforms.Lambda(lambda x: x * 0.00392156862745098),  # Rescale the pixel values
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize the image
])

# Now, you can apply this transform to your dataset
# For example, using torchvision datasets
from torchvision.datasets import ImageFolder

dataset = load_dataset("frgfm/imagenette", 'full_size')

datasets.ImageFolder(root='/path/to/train', transform=train_transform)

# Assuming your images are in a folder called 'data'
# dataset = ImageFolder(root='data', transform=transform)

FileNotFoundError: Found no valid file for the classes MNIST, cifar-10-batches-py. Supported extensions are: .jpg, .jpeg, .png, .ppm, .bmp, .pgm, .tif, .tiff, .webp

In [18]:
print(dataset['train'][0]['image'])

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=500x281 at 0x7F07BA15E2B0>


In [19]:
from transformers import AutoImageProcessor

print('initialize image processor')




initialize image processor


In [24]:
image_processor( dataset['train'][0]['image'] )['pixel_values'][0].shape

(3, 224, 224)

In [25]:
dataset['train'][0]['label'] 

2

In [16]:
import torch
import torch.nn as nn
import torch.distributed as dist
import torch.optim as optim
from torch.utils.data import DataLoader, DistributedSampler
from torchvision.models import resnet50
from torchvision.datasets import ImageNet
from torchvision import transforms
from torch.nn.parallel import DistributedDataParallel as DDP

from transformers import AutoImageProcessor, ResNetModel
import torch
from datasets import load_dataset

import os
import io

from torch.utils.data import Dataset
from transformers import AutoImageProcessor
from PIL import Image
import torch

class CustomImageDataset(Dataset):
    def __init__(self, hf_dataset, transform=None):
        self.hf_dataset = hf_dataset
        self.transform = transform

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        # Get the image and label from the Hugging Face dataset
        item = self.hf_dataset[idx]
        image = item['image']
        # print(image.shape)
        # Check if the image needs to be opened from a bytes-like object
        # if not isinstance(image, Image.Image):
        #     image = Image.open(io.BytesIO(image)).convert("RGB")

        # print(image.shape)
        # Apply the transformation
        # if self.transform:
        image = self.transform(images=image)['pixel_values'][0]

        # Labels can be handled here if needed
        label = item.get('label', torch.tensor(-1))  # Dummy label handling

        return image, label

print('load dataset')
# dataset = load_dataset("imagenet-1k", cache_dir='/home/mheuill/scratch')
test_dataset = dataset['train']  # Select the test split

print('initialize image processor')
image_processor = AutoImageProcessor.from_pretrained("./resnet-50", local_files_only=True)



load dataset
initialize image processor


In [17]:
print('create custom dataset')
custom_dataset = CustomImageDataset(test_dataset, transform=image_processor)


create custom dataset


In [18]:


print('load dataloader')
dataloader = DataLoader(custom_dataset, batch_size=1024,  num_workers=4)

load dataloader


In [3]:
# dataset = load_dataset("imagenet-1k")
# image = dataset["test"]["image"][0]

from transformers import AutoImageProcessor
image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-50")

image_processor.save_pretrained("./resnet-50")
# inputs = image_processor(image, return_tensors="pt")

# model = ResNetModel.from_pretrained("microsoft/resnet-50")



# with torch.no_grad():
#     outputs = model(**inputs)

# last_hidden_states = outputs.last_hidden_state
# list(last_hidden_states.shape)

['./resnet-50/preprocessor_config.json']

In [2]:
image_processor

ConvNextImageProcessor {
  "_valid_processor_keys": [
    "images",
    "do_resize",
    "size",
    "crop_pct",
    "resample",
    "do_rescale",
    "rescale_factor",
    "do_normalize",
    "image_mean",
    "image_std",
    "return_tensors",
    "data_format",
    "input_data_format"
  ],
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_rescale": true,
  "do_resize": true,
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_processor_type": "ConvNextImageProcessor",
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "rescale_factor": 0.00392156862745098,
  "size": {
    "shortest_edge": 224
  }
}

In [14]:
inputs['pixel_values'].shape

torch.Size([1, 3, 224, 224])